In [ ]:
import cherrypy
import json
import redis

redis_client = redis.Redis(
  username='default',
  host='redis-19264.c301.ap-south-1-1.ec2.cloud.redislabs.com',
  port=19264,
  password='6HVuWRH8CSONqykRKS9Y2esqvv6o4II7')

# endpoint /devices
class Devices(object):
    exposed = True

    def GET(self, *path, **query):
        devices = [key.decode().split(':')[0] for key in redis_client.keys('*:power')]
        return json.dumps({'mac_addresses' : devices})

# endpoint /device/{mac_address}
class BatteryStatus(object):
    exposed = True
    
    def GET(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')
        if 'from' not in query:
            raise cherrypy.HTTPError(400, 'Bad Request: missing start time')
        if 'to' not in query:
            raise cherrypy.HTTPError(400, 'Bad Request: missing end time')
        mac_address = path[0]
        start, end = int(query['from']), int(query['to'])
        devices = [key.decode().split(':')[0] for key in redis_client.keys('*:power')]
        if mac_address not in devices:
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address')
        battery_ts = redis_client.ts().range(mac_address + ':battery', start, end)
        power_ts = redis_client.ts().range(mac_address + ':power', start, end)
        if battery_ts and power_ts: 
            timestamps, battery_levels = zip(*battery_ts)
            _, power = zip(*power_ts)
        else:
            timestamps, battery_levels, power = [], [], []
        return json.dumps({
            'mac_address': mac_address,
            'timestamps': timestamps,
            'battery_levels': battery_levels,
            'power_plugged': power
        })
    
    def DELETE(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, 'Bad Request: missing MAC address')
        mac_address = path[0]
        devices = [key.decode().split(':')[0] for key in redis_client.keys('*:power')]
        if mac_address not in devices:
            raise cherrypy.HTTPError(404, 'Not Found: invalid MAC address')
        redis_client.delete(mac_address + ':battery')
        redis_client.delete(mac_address + ':power')
        return 'OK: Everything worked as expected.'
        

conf = {'/': {'request.dispatch': cherrypy.dispatch.MethodDispatcher()}}
cherrypy.tree.mount(Devices(), '/devices', conf)
cherrypy.tree.mount(BatteryStatus(), '/device', conf)
cherrypy.config.update({'server.socket_host': '0.0.0.0'})
cherrypy.config.update({'server.socket_port': 8080})
cherrypy.engine.start()
cherrypy.engine.block()